In [1]:
!pip install torch psutil tqdm numpy rogue-score


ERROR: Could not find a version that satisfies the requirement rogue-score (from versions: none)
ERROR: No matching distribution found for rogue-score

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import torch
import os
import time
import re
import gc
import psutil
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel

def log_to_file(message):
    """Helper function for logging"""
    print(message)

def generate_response(model, tokenizer, instruction, max_length=512, device="cuda"):
    """Generate a response using GPT-2 model directly from instruction"""
    model.eval()

    # Create a prompt for generating response
    input_text = f"[INST] User query: {instruction}\n\nProvide a helpful customer service response: [RESP]"

    # Tokenize input
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
    attention_mask = torch.ones_like(input_ids).to(device)

    try:
        # Generate response
        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=max_length,
                num_beams=5,
                no_repeat_ngram_size=2,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        # Decode the generated text
        generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

        # Extract response part
        if "[RESP]" in generated_text:
            response = generated_text.split("[RESP]")[1].strip()
            # Clean up any trailing tokens
            if "[EOS]" in response:
                response = response.split("[EOS]")[0].strip()
        else:
            # Fallback extraction
            response = generated_text[len(input_text):].strip()
            
        # Format steps if present
        steps_pattern = re.search(r'(\d+)\.\s+([A-Z])', response)
        if steps_pattern or "step" in response.lower() or "follow" in response.lower():
            # Format steps to be on separate lines
            for i in range(1, 10):
                step_marker = f"{i}. "
                if step_marker in response and f"\n{i}. " not in response:
                    response = response.replace(step_marker, f"\n{i}. ")

            # Clean up any excess newlines
            response = re.sub(r'\n\s*\n', '\n\n', response)
            response = response.lstrip('\n')

        # Clean any technical artifacts
        response = re.sub(r'https?://\S+', '', response)  # Remove URLs
        response = re.sub(r'<[^>]*>', '', response)  # Remove HTML tags
        response = re.sub(r'\{\s*"[^"]*":', '', response)  # Remove JSON-like content
        response = re.sub(r'\s+', ' ', response).strip()  # Clean up whitespace

        return response

    except Exception as e:
        print(f"Error in generate_response: {e}")
        return f"I apologize, but I couldn't generate a response. Error: {str(e)}"

# Memory measurement functions
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Convert bytes to MB

def get_peak_memory_usage(func, *args, **kwargs):
    device_param = kwargs.pop('device', None)  # Remove device parameter before calling func
    
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
    
    start_mem = get_memory_usage()
    result = func(*args, **kwargs)  # Call without device parameter
    end_mem = get_memory_usage()
    
    peak_gpu_mem = 0
    if torch.cuda.is_available() and device_param == "cuda":
        peak_gpu_mem = torch.cuda.max_memory_allocated() / 1024 / 1024  # MB
        return result, peak_gpu_mem
    else:
        return result, max(0, end_mem - start_mem)

class DirectGenerationPerformanceTest:
    def __init__(self, model_paths, test_data_path, num_samples=50):
        self.generation_model_path = model_paths["generation_model_path"]
        self.generation_tokenizer_path = model_paths["generation_tokenizer_path"]
        self.test_data_path = test_data_path
        self.num_samples = num_samples
        self.devices = ["cpu", "cuda"] if torch.cuda.is_available() else ["cpu"]
        self.results = {device: [] for device in self.devices}
        self.summary = {device: {"generation_time": [], "memory_usage": []} for device in self.devices}
    
    def load_models(self, device):
        print(f"\nLoading models on {device.upper()}...")
        
        # Load generation model using from_pretrained
        try:
            print(f"Attempting to load generation model from {self.generation_model_path}")
            self.generation_model = GPT2LMHeadModel.from_pretrained(self.generation_model_path).to(device)
            self.generation_tokenizer = GPT2Tokenizer.from_pretrained(self.generation_tokenizer_path)
            self.generation_tokenizer.pad_token = self.generation_tokenizer.eos_token
            self.generation_tokenizer.add_special_tokens({'additional_special_tokens': ['[INST]', '[RESP]', '[EOS]']})
            self.generation_model.resize_token_embeddings(len(self.generation_tokenizer))
        except Exception as e:
            print(f"Error loading generation model: {e}")
            print("Falling back to default GPT2...")
            self.generation_model = GPT2LMHeadModel.from_pretrained('gpt2').to(device)
            self.generation_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
            self.generation_tokenizer.pad_token = self.generation_tokenizer.eos_token
            self.generation_tokenizer.add_special_tokens({'additional_special_tokens': ['[INST]', '[RESP]', '[EOS]']})
            self.generation_model.resize_token_embeddings(len(self.generation_tokenizer))
        
        self.generation_model.eval()
        
        return self.generation_model, self.generation_tokenizer
    
    def run_benchmark(self):
        # Load test data
        with open(self.test_data_path, "r", encoding="utf-8") as f:
            test_data = json.load(f)
            
        # Select samples for testing
        if len(test_data) > self.num_samples:
            test_samples = test_data[:self.num_samples]
        else:
            test_samples = test_data
            
        print(f"Running performance test on {len(test_samples)} samples")
        
        # Run tests on each device
        for device in self.devices:
            print(f"\n{'='*80}")
            print(f"RUNNING BENCHMARK ON {device.upper()}")
            print(f"{'='*80}")
            
            generation_model, generation_tokenizer = self.load_models(device)
            
            for idx, sample in enumerate(tqdm(test_samples, desc=f"Testing on {device.upper()}")):
                instruction = sample["instruction"]
                
                # Measure generation time and memory
                def run_generation():
                    return generate_response(
                        generation_model,
                        generation_tokenizer,
                        instruction,
                        device=device
                    )
                
                generation_start = time.time()
                generated_response, generation_memory = get_peak_memory_usage(
                    run_generation, device=device
                )
                generation_end = time.time()
                generation_time = generation_end - generation_start
                
                # Store results
                result = {
                    "Model": "GPT2-Based Direct Generation",
                    "Text Generation Time (s)": generation_time,
                    "Device": "GPU" if device == "cuda" else "CPU",
                    "Memory Usage (MB)": generation_memory,
                    "Instruction": instruction,
                    "Generated Response": generated_response
                }
                self.results[device].append(result)
                
                # Update summary statistics
                self.summary[device]["generation_time"].append(generation_time)
                self.summary[device]["memory_usage"].append(generation_memory)
                
                # Print sample result
                print(f"\nSample {idx+1}/{len(test_samples)} (Device: {device.upper()}):")
                print(f"Instruction: {instruction}")
                print(f"Generated Response: {generated_response}")
                print(f"Model               | Text Generation Time (s)  | Device     | Memory Usage (MB)")
                print(f"GPT2-Direct Gen     | {generation_time:.4f}                  | {device.upper()}        | {generation_memory:.4f}")
            
            # Calculate average times
            self.summary[device]["avg_generation_time"] = sum(self.summary[device]["generation_time"]) / len(test_samples)
            self.summary[device]["avg_memory_usage"] = sum(self.summary[device]["memory_usage"]) / len(test_samples)
            
            print(f"\n{'='*80}")
            print(f"BENCHMARK SUMMARY FOR {device.upper()}")
            print(f"{'='*80}")
            print(f"Average Generation Time: {self.summary[device]['avg_generation_time']:.4f} seconds")
            print(f"Average Memory Usage: {self.summary[device]['avg_memory_usage']:.4f} MB")
        
        # Save results after all devices are tested
        self.save_results()

    def save_results(self):
        """Save benchmark results to JSON files in the specified format"""
        
        # Save CPU results
        if "cpu" in self.devices and self.results["cpu"]:
            with open("performance_results_direct_cpu.json", "w") as f:
                json.dump(self.results["cpu"], f, indent=4)
            print("CPU results saved to performance_results_direct_cpu.json")
        
        # Save GPU results
        if "cuda" in self.devices and self.results["cuda"]:
            with open("performance_results_direct_gpu.json", "w") as f:
                json.dump(self.results["cuda"], f, indent=4)
            print("GPU results saved to performance_results_direct_gpu.json")
        
        # Create performance log and comparison plots
        self.create_performance_log()
        if "cpu" in self.devices and "cuda" in self.devices:
            self.create_comparison_plots()

    def create_performance_log(self):
        """Create a text file with performance comparisons"""
        with open("direct_generation_performance_log.txt", "w") as f:
            f.write("=== DIRECT GENERATION PERFORMANCE BENCHMARK RESULTS ===\n\n")
            
            # Write CPU results
            if "cpu" in self.devices:
                f.write("CPU PERFORMANCE:\n")
                f.write(f"Average Generation Time: {self.summary['cpu']['avg_generation_time']:.4f} seconds\n")
                f.write(f"Average Memory Usage: {self.summary['cpu']['avg_memory_usage']:.4f} MB\n\n")
            
            # Write GPU results
            if "cuda" in self.devices:
                f.write("GPU PERFORMANCE:\n")
                f.write(f"Average Generation Time: {self.summary['cuda']['avg_generation_time']:.4f} seconds\n")
                f.write(f"Average Memory Usage: {self.summary['cuda']['avg_memory_usage']:.4f} MB\n\n")
            
            # Write comparison
            if "cpu" in self.devices and "cuda" in self.devices:
                f.write("CPU vs GPU COMPARISON:\n")
                gen_speedup = self.summary['cpu']['avg_generation_time'] / self.summary['cuda']['avg_generation_time']
                
                f.write(f"Generation Speed Improvement: {gen_speedup:.2f}x faster on GPU\n\n")
                
                f.write("Notes:\n")
                f.write("- GPU memory usage is typically higher but processing is faster\n")
                f.write("- Direct generation without classification reduces overall processing time\n")
            
            f.write("\n=== END OF REPORT ===\n")
        
        print("Performance log saved to direct_generation_performance_log.txt")

    def create_comparison_plots(self):
        """Create matplotlib comparison plots"""
        import matplotlib.pyplot as plt
        import numpy as np
        
        if "cpu" not in self.devices or "cuda" not in self.devices:
            print("Both CPU and GPU results are needed for comparison plots")
            return
        
        # Set up the plots
        plt.figure(figsize=(12, 8))
        
        # 1. Generation Time Comparison
        plt.subplot(2, 1, 1)
        
        labels = ['Generation Time']
        cpu_times = [self.summary['cpu']['avg_generation_time']]
        gpu_times = [self.summary['cuda']['avg_generation_time']]
        
        x = np.arange(len(labels))
        width = 0.35
        
        plt.bar(x - width/2, cpu_times, width, label='CPU')
        plt.bar(x + width/2, gpu_times, width, label='GPU')
        
        plt.xlabel('Task')
        plt.ylabel('Time (seconds)')
        plt.title('Average Generation Time Comparison')
        plt.xticks(x, labels)
        plt.legend()
        
        # Add value labels on the bars
        for i, v in enumerate(cpu_times):
            plt.text(i - width/2, v + 0.01, f"{v:.2f}s", ha='center')
        
        for i, v in enumerate(gpu_times):
            plt.text(i + width/2, v + 0.01, f"{v:.2f}s", ha='center')
        
        # 2. Memory Usage
        plt.subplot(2, 1, 2)
        
        memory_labels = ['CPU', 'GPU']
        memory_usage = [self.summary['cpu']['avg_memory_usage'],
                      self.summary['cuda']['avg_memory_usage']]
        
        plt.bar(memory_labels, memory_usage)
        plt.xlabel('Device')
        plt.ylabel('Memory Usage (MB)')
        plt.title('Average Memory Usage')
        
        # Add memory usage labels
        for i, v in enumerate(memory_usage):
            plt.text(i, v + 0.1, f"{v:.1f} MB", ha='center')
        
        plt.tight_layout()
        plt.savefig('direct_generation_comparison.png')
        plt.close()
        
        print("Performance comparison plots saved to direct_generation_comparison.png")

def main():
    # Define paths
    model_paths = {
        "generation_model_path": "../../text_generation_results_03-09-25/model",
        "generation_tokenizer_path": "../../text_generation_results_03-09-25/tokenizer"
    }
    
    test_data_path = "../../test.json"
    
    # Verify and print paths
    print(f"Test data path: {test_data_path}")
    
    # Number of samples to test
    num_samples = 50  # You can adjust this number
    
    # Run the benchmark
    benchmark = DirectGenerationPerformanceTest(model_paths, test_data_path, num_samples)
    benchmark.run_benchmark()

if __name__ == "__main__":
    # Check available devices
    # Check available devices
    device_info = "CUDA" if torch.cuda.is_available() else "CPU"
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"Running on: {device_info}")
    
    # Run the benchmark
    main()

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedba

GPU 0: NVIDIA GeForce RTX 3050 Laptop GPU
Running on: CUDA
Test data path: ../../test.json
Running performance test on 50 samples

RUNNING BENCHMARK ON CPU

Loading models on CPU...
Attempting to load generation model from ../../text_generation_results_03-09-25/model


Testing on CPU:   2%|▏         | 1/50 [00:27<22:16, 27.27s/it]


Sample 1/50 (Device: CPU):
Instruction: i do not know what i have to do to get my invoice #85632
Generated Response: i'm sorry to hear that you're unsure about the steps to take in order to retrieve your invoice with the number #{{invoice number}}. let me guide you through the process. first, please log in to your account on our website. once logged in, navigate to the "billing" or "invoices" section. there, you should be able to find a list of your invoices. look for a tab or link that says "view invoice." click on it, and you'll be directed to a page where you can enter the invoice number. if you encounter any difficulties or have further questions, feel free to reach out to our customer support team.
Model               | Text Generation Time (s)  | Device     | Memory Usage (MB)
GPT2-Direct Gen     | 27.2712                  | CPU        | 22.6758


Testing on CPU:   4%|▍         | 2/50 [00:37<13:38, 17.04s/it]

...

Testing on CUDA:  98%|█████████▊| 49/50 [01:58<00:02,  2.21s/it]


Sample 49/50 (Device: CUDA):
Instruction: change  order ord-8335461
Generated Response: i'm sorry to hear that you need to change order number {{order number}}. i understand how important it is for you to have control over your order. to assist you further, could you please provide me with more details about the specific changes you would like to make? this will help me guide you through the process and ensure that your request is handled promptly and accurately.
Model               | Text Generation Time (s)  | Device     | Memory Usage (MB)
GPT2-Direct Gen     | 1.8037                  | CUDA        | 617.3574


Testing on CUDA: 100%|██████████| 50/50 [02:00<00:00,  2.42s/it]


Sample 50/50 (Device: CUDA):
Instruction: i need assistance to send some feedback for ur produucts
Generated Response: thank you for reaching out! i'm here to assist you in sending your feedback about our products and services. your input is incredibly valuable to us as it helps us understand how we can better serve our customers. could you please provide me with more details about the specific product or service you would like to provide feedback on? this will allow me to guide you through the process and ensure that you receive the feedback you deserve.
Model               | Text Generation Time (s)  | Device     | Memory Usage (MB)
GPT2-Direct Gen     | 2.1332                  | CUDA        | 626.7354

BENCHMARK SUMMARY FOR CUDA
Average Generation Time: 2.4145 seconds
Average Memory Usage: 634.2379 MB
CPU results saved to performance_results_direct_cpu.json
GPU results saved to performance_results_direct_gpu.json
Performance log saved to direct_generation_performance_log.txt


Performance comparison plots saved to direct_generation_comparison.png
